In [17]:
import sys
sys.path.append("../../") 

In [18]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.metrics import max_error, mean_squared_error, r2_score
from modelos import *
import pickle

In [19]:
RANDOM_SEED

777

Cargamos el conjunto de datos entero y lo filtramos eliminando las columnas que eliminamos para todos nuestros modelos.

In [20]:
X, y = datos_full()
X_filtrado = X.drop(columns=["Temperatura", 'wspd', 'anio', 'mes', 'dia', 'hora'])
X_filtrado

,moonphase,temp,dewPt,heat_index,rh,pressure,vis,wc,feels_like,uv_index,...,wdir_SSW,wdir_SW,wdir_VAR,wdir_W,wdir_WNW,wdir_WSW,Vviento,PeriodoOlas,Lluvia,Nubosidad
0,0.48,48.0,43.0,48.0,82.0,30.460,6.0,42.5,42.5,0.0,...,0,1,0,0,0,0,14,8,0.0,7
1,0.48,46.0,43.0,46.0,87.0,30.460,6.0,41.0,41.0,0.0,...,0,1,0,0,0,0,12,8,0.0,69
2,0.48,45.0,43.0,45.0,93.0,30.490,6.0,39.0,39.0,0.0,...,0,1,0,0,0,0,10,9,0.0,100
3,0.48,45.0,42.0,45.0,90.0,30.490,6.0,40.0,40.0,0.0,...,0,1,0,0,0,0,9,9,0.0,89
4,0.48,44.0,41.0,44.0,90.0,30.520,6.0,39.5,39.5,0.0,...,0,1,0,0,0,0,9,8,0.0,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12805,0.65,45.5,40.0,45.5,81.0,29.065,6.0,38.5,38.5,0.0,...,0,0,0,0,1,0,23,10,0.2,100
12806,0.65,45.5,40.0,45.5,81.0,29.095,6.0,38.5,38.5,0.0,...,0,0,0,1,0,0,23,10,0.1,100
12807,0.65,44.0,38.0,44.0,81.0,29.155,6.0,35.5,35.5,0.0,...,0,0,0,0,1,0,20,10,0.1,100
12808,0.65,45.0,37.0,45.0,76.0,29.200,6.0,37.0,37.0,0.0,...,0,0,0,0,1,0,20,10,0.1,100


Necesitamos escalar estos datos, pero necesitamos recuperar la escala que habíamos aplicado al conjunto de entrenamiento.

Cargamos el conjunto de datos de train y hacemos fit para ajustar la escala. Más tarde esta escala la usaremos con el conjunto de datos entero.

In [21]:
# Para el modelo con todas las variables
X_train, X_test, y_train, y_test = sep_train_test()
# Eliminamos las columnas repetidas y las columnas temporales (visto en el notebook del analisis)
X_train = X_train.drop(columns=["Temperatura", 'wspd', 'anio', 'mes', 'dia', 'hora'])

In [22]:
# Para probar con datos escalados
scaler = ColumnTransformer(
    transformers=[
        # no escalonamos las dummy de dirección de viento
        ("num", StandardScaler(), [c for c in X_train.columns if not c.startswith('wdir_')]),
    ],
    remainder='passthrough'
)
scaler.fit(X_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('num', StandardScaler(),
                                 ['moonphase', 'temp', 'dewPt', 'heat_index',
                                  'rh', 'pressure', 'vis', 'wc', 'feels_like',
                                  'uv_index', 'Vviento', 'PeriodoOlas',
                                  'Lluvia', 'Nubosidad'])])

Transformamos el conjunto de datos entero.

In [24]:
scaler.transform(X_filtrado);

Ya podemos entrenar el modelo con los hiperparámetros óptimos con el conjunto de datos completo.

Ponemos los parámetros óptimos a mano.

In [25]:
modelo_def = RandomForestRegressor(random_state = RANDOM_SEED, n_estimators = 201, min_samples_split = 4, min_samples_leaf = 4,
                                   max_depth = 28, bootstrap = True)
modelo_def

RandomForestRegressor(max_depth=28, min_samples_leaf=4, min_samples_split=4,
                      n_estimators=201, random_state=777)

Ajustamos el modelo.

In [26]:
modelo_def.fit(X_filtrado,y)

RandomForestRegressor(max_depth=28, min_samples_leaf=4, min_samples_split=4,
                      n_estimators=201, random_state=777)

Hacemos predicciones sobre los datos de entrenamiento, algo que no tiene mucho sentido pero va a servir para comparar que el modelo se guarda bien en formato pickle

In [31]:
predicciones_modelo_def_antes = modelo_def.predict(X_filtrado)

Guardamos el modelo en formato pickle

In [32]:
filename = 'modelo_rf_def.pkl'  # Elige el nombre de archivo que prefieras
with open(filename, 'wb') as file:
    pickle.dump(modelo_def, file)

Cargamos el modelo guardado como .pickle

In [33]:
filename = 'modelo_rf_def.pkl'  # Elige el nombre de archivo que prefieras
with open(filename, 'rb') as file:
    model = pickle.load(file)

model

RandomForestRegressor(max_depth=28, min_samples_leaf=4, min_samples_split=4,
                      n_estimators=201, random_state=777)

Hacemos predicciones sobre el mismo conjunto de entrenamiento

In [35]:
predicciones_modelo_def_despues = model.predict(X_filtrado)

Comprobamos que las predicciones son iguales

In [36]:
(predicciones_modelo_def_antes != predicciones_modelo_def_despues).sum()

0

El modelo es el mismo antes y despúes de guardarlo.